In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
import matplotlib as plt
import matplotlib.pylab as plt
import seaborn as sns

In [45]:
t['label'].unique()

array([0])

In [46]:
q = '''
SELECT CASE
           WHEN c.imp_id IS NOT NULL THEN 1
           ELSE 0
       END AS label,
       '20200815' AS DAY,
       i.hour AS hour,
       i.content_id AS imp_content_id,
       i.content_position AS imp_content_position,
       i.page_id AS imp_page_id,
       i.ref_page_id AS imp_ref_page_id,
       i.ref_term AS imp_ref_term,
       i.ref_source AS imp_ref_source,
       i.user_id AS imp_user_id,
       i.content_owner AS imp_content_owner
FROM
  (SELECT imp_id,
          content_id,
          content_position,
          page_id,
          ref_page_id,
          ref_term,
          ref_source,
          user_id,
          content_owner,
          YEAR,
          MONTH,
          DAY,
          hour
   FROM bun_log_db.app_event_type_impression
   WHERE year||month||day||hour BETWEEN '2020081500' AND '2020081503'
     AND ref_source IN ('power_up',
                        'ad_super_up',
                        'ad_super_up_shop')
     AND app_name != '번개장터 DEBUG'
     AND app_id != 'kr.co.quicket.debug'
     AND content_type = 'product') AS i
LEFT JOIN
  (SELECT imp_id,
          content_id,
          content_position,
          page_id,
          ref_page_id,
          ref_term,
          ref_source,
          user_id,
          content_owner,
          YEAR,
          MONTH,
          DAY,
          hour
   FROM bun_log_db.app_event_type_view
   WHERE year||month||day||hour BETWEEN '2020081500' AND '2020081503'
     AND ref_source IN ('power_up',
                        'ad_super_up',
                        'ad_super_up_shop')
     AND app_name != '번개장터 DEBUG'
     AND app_id != 'kr.co.quicket.debug'
     AND content_type = 'product') AS c ON i.imp_id=c.imp_id
'''
imp = pd.read_sql(q, con=bun_dw)

In [47]:
print('label of 1:', imp[imp['label'] == 1].shape[0])
print('label of 0:', imp[imp['label'] == 0].shape[0])

label of 1: 30197
label of 0: 964130


In [48]:
user_id_list = ','.join('\''+ str(i) +'\'' for i in imp['imp_user_id'].unique())
content_owner_list = ','.join('\''+ str(i) +'\'' for i in imp['imp_content_owner'].unique())
content_id_list = ','.join('\''+ str(i) +'\'' for i in imp['imp_content_id'].unique())

In [49]:
q = '''
SELECT u.id,
      ue.sex as user_sex,
      Coalesce(NULL, Ceil((Extract(DAY
                                   FROM Getdate() - ue.birthdate :: timestamptz) / 365.25)), 0) AS user_age,
      ue.following_cnt as user_following_cnt,
      ue.bunpay_count as user_bunpay_count
FROM service1_quicket.user_ u
JOIN service1_quicket.user_extra_info ue ON u.id = ue.uid
WHERE ue.is_identification = 1
 AND ue.birthdate BETWEEN '1930-01-01' AND '2010-12-31'
 AND u.id in ({})
'''.format(user_id_list)
user_ids = pd.read_sql(q, con=bun_dw)

DataError: (psycopg2.errors.InvalidTextRepresentation) invalid input syntax for integer: "None"

[SQL: 
SELECT u.id,
      ue.sex as user_sex,
      Coalesce(NULL, Ceil((Extract(DAY
                                   FROM Getdate() - ue.birthdate :: timestamptz) / 365.25)), 0) AS user_age,
      ue.following_cnt as user_following_cnt,
      ue.bunpay_count as user_bunpay_count
FROM service1_quicket.user_ u
JOIN service1_quicket.user_extra_info ue ON u.id = ue.uid
WHERE ue.is_identification = 1
 AND ue.birthdate BETWEEN '1930-01-01' AND '2010-12-31'
 AND u.id in ('11954212','5463242','8339028','3340828','5872479','3367511','12073198','3162303','7921212','2947479','5027850','4675871','11308861','10722306','2368740','7833342','10670741','609432','7103861','10221110','11611981','2271217','11835375','9586091','412762','10745284','11498942','11953283','7896246','6482165','3206935','299737','5333740','5837012','1384776','11439121','6124796','4034199','3513607','4103904','2773884','7541638','1

In [29]:
q = '''
SELECT u.id,
      u.grade as owner_grade,
      u.item_count as owner_item_count,
      u.interest as owner_interest,
      ue.follower_cnt as owner_follower_cnt,
      ue.bunpay_count as owner_bunpay_count
FROM service1_quicket.user_ u
JOIN service1_quicket.user_extra_info ue ON u.id = ue.uid
WHERE ue.is_identification = 1 
 AND u.id in ({})
'''.format(content_owner_list)
content_owners = pd.read_sql(q, con=bun_dw)

In [31]:
q = '''
SELECT p.id,
      p.price as content_price,
      left(p.category_id, 6) as content_category_id,
      pe.emergency_cnt as content_emergency_cnt,
      pe.comment_cnt as content_comment_cnt,
      pe.interest as content_interest,
      pe.pfavcnt as content_pfavcnt
FROM service1_quicket.product_info p
JOIN service1_quicket.product_ext pe ON p.id = pe.pid
WHERE p.price BETWEEN 10000 AND 10000000
 AND p.id in ({})
'''.format(content_id_list)
content_ids = pd.read_sql(q, con=bun_dw)

In [32]:
imp['imp_user_id'] = imp['imp_user_id'].astype(str)
imp['imp_content_owner'] = imp['imp_content_owner'].astype(str)
imp['imp_content_id'] = imp['imp_content_id'].astype(str)
user_ids['id'] = user_ids['id'].astype(str)
content_owners['id'] = content_owners['id'].astype(str)
content_ids['id'] = content_ids['id'].astype(str)



df = pd.merge(imp, user_ids, how='left', left_on='imp_user_id', right_on='id', suffixes=('_left', '_right'))
df = pd.merge(df, content_owners, how='left', left_on='imp_content_owner', right_on='id')
df = pd.merge(df, content_ids, how='left', left_on='imp_content_id', right_on='id')
print(df[df['imp_user_id'] == df['imp_content_owner']].shape[0])
df = df[df['imp_user_id'] != df['imp_content_owner']]

In [34]:
print('label of 1:', df[df['label'] == 1].shape[0])
print('label of 0:', df[df['label'] == 0].shape[0])

label of 1: 0
label of 0: 1044591
